In [1]:

## UNCOMMENT THIS CELL IF USING GOOGLE COLAB ###

from google.colab import drive

drive.mount('/content/drive', force_remount=True)

# Enter the path in your Drive..
FOLDERNAME = "Transformer"

assert FOLDERNAME is not None, "[!] Enter the foldername."

# Now that we've mounted your Drive, this ensures that
# the Python interpreter of the Colab VM can load
# python files from within it.
import sys
sys.path.append(f'/content/drive/My Drive/{FOLDERNAME}')

path = f'/content/drive/My Drive/{FOLDERNAME}'

Mounted at /content/drive


In [2]:
%load_ext autoreload
%autoreload 2

!pip install transformers

import os
import gc
import torch
import numpy as np
from torch import nn
from torch.utils.data import DataLoader
from utils2 import *
from dataset import *
from model import *
from Scheduler import *


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# device = "cpu"
print(device)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 40.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 22.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 83.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 59.7 MB/s eta 0:00:00
cuda:0


In [3]:
data_folder = "WMT14 EN-GE/"

vocab_folder = os.path.join(data_folder,"vocab")
train_folder = os.path.join(data_folder,"train")
test_folder = os.path.join(data_folder,"test")
mappings_path = os.path.join(data_folder,"dict.en-de")

folders_map = {"train":train_folder,"test":test_folder}

In [4]:
!pip install wget
import wget

for folder in [data_folder, vocab_folder, train_folder, test_folder]:
  if not(os.path.exists(folder)):
    os.mkdir(folder)

urls_to_load = {"https://nlp.stanford.edu/projects/nmt/data/wmt14.en-de/vocab.50K.en":vocab_folder,
                "https://nlp.stanford.edu/projects/nmt/data/wmt14.en-de/vocab.50K.de":vocab_folder,
                "https://nlp.stanford.edu/projects/nmt/data/wmt14.en-de/train.en":train_folder,
                "https://nlp.stanford.edu/projects/nmt/data/wmt14.en-de/newstest2012.en":test_folder,
                "https://nlp.stanford.edu/projects/nmt/data/wmt14.en-de/train.de":train_folder,
                "https://nlp.stanford.edu/projects/nmt/data/wmt14.en-de/newstest2012.de":test_folder}  # a list to store the urls

files_namings = {"https://nlp.stanford.edu/projects/nmt/data/wmt14.en-de/vocab.50K.en":"vocab.en",
                "https://nlp.stanford.edu/projects/nmt/data/wmt14.en-de/vocab.50K.de":"vocab.de",
                "https://nlp.stanford.edu/projects/nmt/data/wmt14.en-de/train.en":"train.en",
                "https://nlp.stanford.edu/projects/nmt/data/wmt14.en-de/newstest2012.en":"test.en",
                "https://nlp.stanford.edu/projects/nmt/data/wmt14.en-de/train.de":"train.de",
                "https://nlp.stanford.edu/projects/nmt/data/wmt14.en-de/newstest2012.de":"test.de"}

# starting to download
print("Starting downloading")
for url,dir in urls_to_load.items():
    file_name = dir + '/' + files_namings[url] # get the full path to the file
    if not(os.path.exists(file_name)):
     # if doesn't exists, download it.
        file_name = wget.download(url, out=file_name)
        print(file_name)

print("ok")

  Preparing metadata (setup.py) ... done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9656 sha256=c54b1af28ff1281b4e01ff40ece6e58baaac7c7582a489184e8e6e06775fe031
  Stored in directory: /root/.cache/pip/wheels/8b/f1/7f/5c94f0a7a505ca1c81cd1d9208ae2064675d97582078e6c769
Successfully built wget
Starting downloading
WMT14 EN-GE/vocab/vocab.en
WMT14 EN-GE/vocab/vocab.de
WMT14 EN-GE/train/train.en
WMT14 EN-GE/test/test.en
WMT14 EN-GE/train/train.de
WMT14 EN-GE/test/test.de
ok


In [5]:
data_map = {"en":{"train":[],"test":[],"val":[]},
             "de":{"train":[],"test":[],"val":[]}}

data_text = extractTextFromFolders(folders_map,data_map,val_split = 0.75, limit = 5e4)

extracting text from folders ...:   0%|          | 0/2 [00:00<?, ?it/s]

In [6]:
tokenizers = create_train_bpe_tokenizer(folders_map,data_map, tokenizers_path = os.path.join(path,f"tokenizers"))

training tokenizer per language ...:   0%|          | 0/2 [00:00<?, ?it/s]

True /content/drive/My Drive/Transformer/tokenizers/tokenizer-en.json
loading pretrained  ..../content/drive/My Drive/Transformer/tokenizers/tokenizer-en.json
True /content/drive/My Drive/Transformer/tokenizers/tokenizer-de.json
loading pretrained  ..../content/drive/My Drive/Transformer/tokenizers/tokenizer-de.json


In [7]:
tokenizer_en, tokenizer_de = tokenizers["en"], tokenizers["de"]

In [8]:
tokenizer_en.decode(tokenizer_en.encode_batch(["Hi !", "How are you gneingieinhgeinh ?"],)[1].ids)

'How are you gne ing ie in h ge in h ?'

In [9]:
data_tokens = extractTokens(data_text,tokenizers)

extracting tokens ...:   0%|          | 0/2 [00:00<?, ?it/s]

In [10]:
data_encodings = extractEncodings(data_tokens,tokenizers)

extracting encodings ...:   0%|          | 0/2 [00:00<?, ?it/s]

In [11]:
print(tokenizer_en.decode(data_encodings["en"]["train"][5155].tolist()))
print(tokenizer_de.decode(data_encodings["de"]["train"][5155].tolist()))

We are available from mon day to sat ur day from 8 . 30 until 23 . 00 on + 31 ( 0 ) 38 ## AT ##-## AT ## 33 3 01 01 .
Wir sind von Montag bis Samstag von 08 . 30 bis 23 . 00 Uhr zu erreichen unter Nummer + 49 30 ## AT ##-## AT ## 8 96 77 9 9 19 .


In [12]:
wmt_train = WMT(inpt_encodings=data_encodings["en"]["train"],tgt_encodings=data_encodings["de"]["train"],
                eos_index = tokenizer_en.token_to_id("[EOS]"), pad_index = tokenizer_en.token_to_id("[PAD]"))
wmt_val = WMT(inpt_encodings=data_encodings["en"]["val"],tgt_encodings=data_encodings["de"]["val"],
              eos_index = tokenizer_en.token_to_id("[EOS]"), pad_index = tokenizer_en.token_to_id("[PAD]"))
wmt_test = WMT(inpt_encodings=data_encodings["en"]["test"],tgt_encodings=data_encodings["de"]["test"],
               eos_index = tokenizer_en.token_to_id("[EOS]"), pad_index = tokenizer_en.token_to_id("[PAD]"))

In [13]:
gc.collect()

0

In [14]:
batch_size = 64
trainLoaders = {}

train_dataloader = DataLoader(wmt_train, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(wmt_val, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(wmt_test, batch_size=batch_size, shuffle=True)


trainLoaders["train"] = train_dataloader
trainLoaders["val"] = val_dataloader


## Generating an example ##
for i,batch in enumerate(iter(val_dataloader)):

    inputs,targets = batch["input"],batch["target"]
    idx = np.random.randint(0,len(inputs))
    if i < 1:
        print(f"inputs batch shape: {inputs['encodings'].size()}")
        print(f"targets batch shape: {targets['decoder_input_encodings'].size()}")
        sample = (inputs["encodings"][idx],targets["decoder_input_encodings"][idx])
        print(f"encodings : {sample[0]}")
        print(f"masks : {inputs['masks'][idx]}")
        print(f"masks : {inputs['masks'].shape}")
        print(f"masks : {targets['masks'][idx]}")
        print(f"masks : {targets['masks'].shape}")
        print(tokenizer_en.decode(sample[0].tolist()),"\n",
             tokenizer_de.decode(sample[1].tolist()))
        break

inputs batch shape: torch.Size([64, 210])
targets batch shape: torch.Size([64, 241])
encodings : tensor([ 5374,  3002,  2990,  8290,  3038,  9059,  3053,  4708, 23470,    15,
         4383,  3121,  3044,  4896,  3038,  3710,  3003, 16380,  4383,  3004,
        12021,  3244,  7196,  3300,  3304,    17,     6,     3,     3,     3,
            3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
            3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
            3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
            3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
            3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
            3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
            3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
            3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
            3,     3,     3,     3,     3,   

In [15]:
model = Transformer(src_vocabSize = tokenizer_en.get_vocab_size(),
                    tgt_vocabSize = tokenizer_de.get_vocab_size(),
                    src_max_len = data_encodings["en"]["train"].shape[1],
                    tgt_max_len = data_encodings["de"]["train"].shape[1],
                    d_embed = 512,
                    d_model = 512,
                    d_ff = 2056,
                    dropout = 0.1,
                    noEncoder = 1,
                    noDecoder = 1,
                    pad_index = tokenizer_en.token_to_id("[PAD]"),
                   device = device).to(device)

In [16]:
optimizer = torch.optim.Adam(model.parameters(),
                             betas = (0.9, 0.98),
                              eps = 1.0e-9)

In [17]:
scheduler = Scheduler(optimizer =  optimizer,
                 dim_embed= 512,
                 warmup_steps=450)

In [18]:
print(f"Numeber of the model's trainable paramaters : {count_parameters(model)}")


Numeber of the model's trainable paramaters : 53477696


In [28]:
## An example of the untrained model prediction ##

out = model(wmt_train[:1]["input"], wmt_train[:1]["target"])
print(out.shape)
print(f"True label : {tokenizer_de.decode(wmt_train[:1]['target']['target_encodings'][0].tolist())}")
print(f"Prediction : {tokenizer_de.decode(torch.argmax(out,2)[0].tolist())}")

## The prediction loss ##
loss = translationLoss(output=out, target=wmt_train[:1]["target"]["target_encodings"].to(device), pad_index = tokenizer_en.token_to_id("[PAD]"), label_smoothing = 0.2)
print(loss)

torch.Size([1, 241, 30000])
True label : iron ce ment ist eine gebrauch s ## AT ##-## AT ## fertige Pa ste , die mit einem Sp ach tel oder den Fin gern als Hoh l ke hle in die For me cken ( Winkel ) der Stahl guss - Kok ille aufge tragen wird .
Prediction : Ein ce of . ein Auswahl tem . AT ##-## AT ## AT Pa ste , die von einem kleinen ag es oder ein ( stern oder ein l & pt ) der von est oder en Sp ( . Stahl ku ) oder ille des tragen . . . . . . ) . ) . . . . . ) ) . . . . . . . . . . . . ) ) ) ) . . . ) ) . . . . . . ) . . ) ) ) . . ) ) ) ) ) . ) ) ) ) ) ) ) ) ) ) ) ) ) . . ) ) . . . . . . . z . . . ) ) . ) ) ) ) . ) ) . . ) ) ) . ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) . . ) ) ) ) . ) ) ) ) ) ) ) ) ) ) ) . . . ) ) ) . . . . ) ) ) ) ) ) . . ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) )
tensor(5.2782, device='cuda:0', grad_fn=<AddBackward0>)


In [20]:
if not (os.path.exists("trained")):
        os.mkdir("trained")

filename = "transformer-weights"

In [23]:
trainedModel = train_model(model, trainLoaders, translationLoss, optimizer,
                           pad_index = tokenizer_en.token_to_id("[PAD]"), label_smoothing = 0.2,
                           scheduler = scheduler, num_epochs=10, device = device, isSave=True,
                           filename = filename, verbose = False)
!cp -r './trained/transformer-weights' /content/drive/MyDrive/

Epoch 0/9
----------


Predicting ...:   0%|          | 0/586 [00:00<?, ?it/s]

train prev epoch Loss: 0
train current epoch Loss: 10.856038148899534, lr = 0.0005504098336278543


Predicting ...:   0%|          | 0/196 [00:00<?, ?it/s]

val prev epoch Loss: 0
val current epoch Loss: 14.071060696426702, lr = 0.0005504098336278543
Epoch 1/9
----------


...

val prev epoch Loss: 14.141753420537832
val current epoch Loss: 14.142788648605347, lr = 0.0004188127637955846
Epoch 9/9
----------


Predicting ...:   0%|          | 0/586 [00:00<?, ?it/s]

train prev epoch Loss: 10.437339624854078
train current epoch Loss: 10.397767708163213, lr = 0.0004082091042707


Predicting ...:   0%|          | 0/196 [00:00<?, ?it/s]

val prev epoch Loss: 14.142788648605347
val current epoch Loss: 14.162149361201696, lr = 0.0004082091042707

Training complete in 87m 27s
Best val loss: 14.071061


In [25]:
results = evaluate_model(model, train_dataloader, tokenizer_de, device = device)

Predicting ...:   0%|          | 0/586 [00:00<?, ?it/s]

 Result : 0.1134713739156723

Evaluating complete in 2m 57s


In [21]:
model.load_state_dict(torch.load(os.path.join(path,os.path.join("trained",filename))))

<All keys matched successfully>

In [ ]:
torch.cuda.empty_cache()
gc.collect()

0

In [26]:
for i,batch in enumerate(iter(train_dataloader)):

    inputs,targets = batch["input"],batch["target"]
    idx = np.random.randint(0,len(inputs))
    if i < 1:
        print(f"inputs batch shape: {inputs['encodings'].size()}")
        print(f"targets batch shape: {targets['decoder_input_encodings'].size()}")
        sample = (inputs["encodings"][0],targets["decoder_input_encodings"][0])

        print(f"Input English Sentence : {tokenizer_en.decode(sample[0].tolist())}")
        print(f"Input German Sentence (Translated) : {tokenizer_de.decode(sample[1].tolist())}")

        out = model(inputs, targets)
        print(out.shape)

        print(f"Predicted German Sentence (Translated) : {tokenizer_de.decode(torch.argmax(out[idx],1).tolist())}")

        loss = translationLoss(output = out, target = targets['target_encodings'].to(device), pad_index = tokenizer_de.token_to_id("[PAD]"), label_smoothing = 0.2)
        print(loss)

        bleu_score = score(out, targets, tokenizer_de, kind = "bleu")
        print(bleu_score)

        break



inputs batch shape: torch.Size([64, 210])
targets batch shape: torch.Size([64, 241])
Input English Sentence : Holiday Houses &# 124 ; Rural Hom es &# 124 ; Apartments &# 124 ; Hotels &# 124 ; Hostel &# 124 ; Things To Do &# 124 ; Last Minute Offers !
Input German Sentence (Translated) : Ferienhäuser &# 124 ; Land häuser &# 124 ; Ferienwohnungen &# 124 ; Hotels &# 124 ; Pensionen &# 124 ; Urlaub &# 124 ; Last Minute Angebote ! !
torch.Size([64, 241, 30000])
Predicted German Sentence (Translated) : Das lo ved International Airport , Restaurants Garden a wick , , Restaurants able , ain Gallery , Restaurants f . , Pubs Sehenswürdigkeiten Sehenswürdig keit , Victoria die erreichbar Hotel aus dem erreichen Sie alle Schritte Theater , Restaurants , Restaurants bars , Pubs . . . . . . . . . . . . . . . . . . . . . . . . . . . entfernt . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . und und und . . . . . . . . . . . . . .

In [ ]:
!cp -r './trained/transformer-weights' /content/drive/MyDrive/

In [ ]:
" ".join(['further', ',', 'skype', 'reserves', 'the', 'right', 'to', 'use', 'email', 'or', 'the', 'skype', 'software', 'to', 'notify', 'you', 'of', 'any', 'eventual', 'claims', 'related', 'to', 'your', 'use', 'of', 'skype', 'software', ',', 'the', 'skype', 'websites', 'and', '/', 'or', 'skype', 'products', ',', 'including', 'without', 'limitation', 'claims', 'of', 'violation', 'of', 'third', 'party', 'rights', '.', '<EOS>'])

'further , skype reserves the right to use email or the skype software to notify you of any eventual claims related to your use of skype software , the skype websites and / or skype products , including without limitation claims of violation of third party rights . <EOS>'

In [31]:
infer(model.to(device), "fix my computer", tokenizers, device = device)

Input English Sentence : fix my computer
input_masks : torch.Size([1, 1, 210])
input_masks : tensor([[[ True,  True,  True,  True, False, False, False, False, False, False,
          False, False, False, False, False, False, False, False, False, False,
          False, False, False, False, False, False, False, False, False, False,
          False, False, False, False, False, False, False, False, False, False,
          False, False, False, False, False, False, False, False, False, False,
          False, False, False, False, False, False, False, False, False, False,
          False, False, False, False, False, False, False, False, False, False,
          False, False, False, False, False, False, False, False, False, False,
          False, False, False, False, False, False, False, False, False, False,
          False, False, False, False, False, False, False, False, False, False,
          False, False, False, False, False, False, False, False, False, False,
          False, False, Fal

'Rechner install ierter Computer meine Computer meine Computer meine Computer meine Computer meine Computer meine Computer meine Computer meine Computer meine Computer meine Computer meine Computer meine Computer meine Computer meine Computer meine Computer meine Computer meine Computer meine Computer meine Computer meine Computer meine Computer meine Computer meine Computer meine Computer meine Computer meine Computer meine Computer meine Computer meine Computer meine Computer meine Computer meine Computer meine Computer meine Computer meine Computer meine Computer meine Computer meine Computer meine Computer meine Computer meine Computer meine Computer meine Computer meine Computer meine Computer meine Computer meine Computer meine Computer meine Computer meine Computer meine Computer meine Computer meine Computer meine Computer meine Computer meine Computer meine Computer meine Computer meine Computer meine Computer meine Computer meine Computer meine Computer meine Computer meine C